In [ ]:
!pip3 install shapely==1.7.1

In [ ]:
!pip install google-cloud-aiplatform
!pip install openai
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.0/321.0 kB 11.5 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.7 MB/s eta 0:00:00


In [ ]:
from langchain.llms import VertexAI
from langchain import PromptTemplate, LLMChain

In [ ]:
import os

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/ucr-ursa-major-maverics-lab-5986748a3acf.json"


In [ ]:
import pandas as pd


In [ ]:
train=pd.read_csv("/content/extractive_train.csv")

In [ ]:
a = train['Original Article'].iloc[0]

In [ ]:
from __future__ import annotations

from google.auth import default
import pandas as pd
import vertexai
from vertexai.preview.language_models import TextGenerationModel
credentials, _ = default(scopes=["https://www.googleapis.com/auth/cloud-platform"])

def tuning(
    project_id: str,
    location: str,
    training_data: pd.DataFrame | str,
    train_steps: int = 300,
) -> None:

    vertexai.init(project=project_id, location=location, credentials=credentials)

    model = TextGenerationModel.from_pretrained("text-bison@001")


    model.tune_model(
        training_data=training_data,
        train_steps=train_steps,
        tuning_job_location="europe-west4",
        tuned_model_location = "us-central1",
        )
    if isinstance(training_data, pd.DataFrame):
            training_data['input_text'] = "I am going to provide you lots of examples of articles, their respective highlights. The output is going to be a summary generated by a small model and it's rogue score. For the rogue score, higher rogue score means that it is a better summary" + training_data['input_text']
            #training_data["input_text"] = "Summarize the provided article. An output with a high Rouge score, a value between 0 and 1, indicates a better summary. Original Article: " + training_data["Original Article"] + " Output: Summary: " + training_data["Generated Summary"] + " Rouge Score: " + training_data["R1-F"]
    print(model._job.status)

# project_id = "ucr-ursa-major-maverics-lab"
# location = "us-central1"

# gcs_uri = "gs://textsummarisation/extractive_train.jsonl"

# tuning(project_id, location, gcs_uri)
# Load the local JSONL file into a pandas DataFrame
local_jsonl_path = "/content/extractive_train.jsonl"
df = pd.read_json(local_jsonl_path, lines=True)

# Call the tuning function with the DataFrame
project_id = "ucr-ursa-major-maverics-lab"
location = "us-central1"
tuning(project_id, location, df)



Forbidden: ignored

In [ ]:
from __future__ import annotations


from google.auth import default
import pandas as pd
import vertexai
from vertexai.preview.language_models import TextGenerationModel

credentials, _ = default(scopes=["https://www.googleapis.com/auth/cloud-platform"])


def tuning(
    project_id: str,
    location: str,
    training_data: pd.DataFrame | str,
    train_steps: int = 300,
) -> None:
    """Tune a new model, based on a prompt-response data.

    "training_data" can be either the GCS URI of a file formatted in JSONL format
    (for example: training_data=f'gs://{bucket}/{filename}.jsonl'), or a pandas
    DataFrame. Each training example should be JSONL record with two keys, for
    example:
      {
        "input_text": <input prompt>,
        "output_text": <associated output>
      },
    or the pandas DataFame should contain two columns:
      ['input_text', 'output_text']
    with rows for each training example.

    Args:
      project_id: GCP Project ID, used to initialize vertexai
      location: GCP Region, used to initialize vertexai
      training_data: GCS URI of jsonl file or pandas dataframe of training data
      train_steps: Number of training steps to use when tuning the model.
    """
    vertexai.init(project=project_id, location=location, credentials=credentials)
    model = TextGenerationModel.from_pretrained("text-bison@001")

    model.tune_model(
        training_data=training_data,
        # Optional:
        train_steps=train_steps,
        tuning_job_location="europe-west4",
        tuned_model_location="us-central1",
    )

    print(model._job.status)

In [ ]:
project_id = "ucr-ursa-major-maverics-lab"
location = "us-central1"

# Sample DataFrame for illustration
df = pd.DataFrame({
    'input_text': ['Example input 1', 'Example input 2'],
    'output_text': ['Example output 1', 'Example output 2']
})

tuning(project_id, location, df)


Forbidden: ignored

In [ ]:
from __future__ import annotations


from google.auth import default
import pandas as pd
import vertexai
from vertexai.preview.language_models import TextGenerationModel

llm = VertexAI(
        project='ucr-ursa-major-maverics-lab',
        model_name="text-bison@001",
        temperature=0.2,
        max_output_tokens=1024,
        top_p=0.8,
        top_k=40
    )

llm.run(
        training_data=['input_text', 'output_text'],
        # Optional:
        train_steps=100,
        tuning_job_location="europe-west4",
        tuned_model_location="https://us-central1-aiplatform.googleapis.com",
    )


AttributeError: ignored

In [ ]:
import vertexai
from vertexai.language_models import TextGenerationModel


def myfunc(
    temperature: float,
    project_id: str,
    location: str,
    a: str,
) -> str:
    vertexai.init(project=project_id, location=location)
    # TODO developer - override these parameters as needed:
    parameters = {
        "temperature": temperature,  # Temperature controls the degree of randomness in token selection.
        "max_output_tokens": 1024,  # Token limit determines the maximum amount of text output.
        "top_p": 0.8,  # Tokens are selected from most probable to least until the sum of their probabilities equals the top_p value.
        "top_k": 40,  # A top_k of 1 means the selected token is the most probable among all tokens.
    }

    model = TextGenerationModel.from_pretrained("text-bison@001")
    response = model.predict(
        f"""Summarize this {a}.""",
        **parameters,
    )
    print(f"Response from Model: {response.text} ")

In [ ]:
project_id_value = "ucr-ursa-major-maverics-lab"
location_value = "us-central1"
myfunc(0.2, project_id_value, location_value,a)


Response from Model: The Germanwings co-pilot Andreas Lubitz had a history of depression and had previously been treated for suicidal tendencies, according to the airline's CEO Carsten Spohr.

Spohr said that Lubitz had informed the airline of his condition in 2009, and that he had been cleared to fly after undergoing treatment.

However, the airline's chief medical officer said that Lubitz had not disclosed the full extent of his condition, and that he had not been fully fit to fly.

The airline has since suspended all of its flights with Lubitz as the co-pilot. 


In [ ]:
interview(article)

TypeError: ignored